___

<a href='http://www.pieriandata.com'><img src='../Pierian_Data_Logo.png'/></a>
___
<center><em>Copyright by Pierian Data Inc.</em></center>
<center><em>For more information, visit us at <a href='http://www.pieriandata.com'>www.pieriandata.com</a></em></center>

# Keras-RL DQN Exercise


In this exercise you are going to implement your first keras-rl agent based on the **Acrobot** environment (https://gym.openai.com/envs/Acrobot-v1/) <br />
The goal of this environment is to maneuver the robot arm upwards above the line with as little steps as possible

**TASK: Import necessary libraries** <br />

In [1]:
import gym
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam
from rl.agents.dqn import DQNAgent

**TASK: Create the environment** <br />
The name is: *Acrobot-v1*

In [2]:
env_name = "Acrobot-v1"
env = gym.make(env_name)

In [3]:
num_actions = env.action_space.n
num_observations = env.observation_space.shape
print(f"Action Space: {env.action_space.n}")
print(f"Observation Space: {num_observations}")

assert num_actions == 3 and num_observations == (6,) , "Wrong environment!"

Action Space: 3
Observation Space: (6,)


**TASK: Create the Neural Network for your Deep-Q-Agent** <br />
Take a look at the size of the action space and the size of the observation space.
You are free to chose any architecture you want! <br />
Hint: It already works with three layers, each having 64 neurons.

In [4]:
model = Sequential()

model.add(Flatten(input_shape=(1, ) + num_observations))

model.add(Dense(64))
model.add(Activation("relu"))

model.add(Dense(64))
model.add(Activation("relu"))

model.add(Dense(64))
model.add(Activation("relu"))


model.add(Dense(num_actions))
model.add(Activation("linear"))

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 6)                 0         
_________________________________________________________________
dense (Dense)                (None, 64)                448       
_________________________________________________________________
activation (Activation)      (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
activation_1 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
activation_2 (Activation)    (None, 64)                0

**TASK: Initialize the circular buffer**<br />
Make sure you set the limit appropriately (50000 works well)

In [6]:
from rl.memory import SequentialMemory

In [7]:
memory = SequentialMemory(limit=50000, window_length=1)

**TASK: Use the epsilon greedy action selection strategy with *decaying* epsilon**

In [8]:
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [9]:
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(),
                              attr="eps",
                              value_max=1.0,
                              value_min=0.1,
                              value_test=0.05,
                              nb_steps=150000
                             )

**TASK: Create the DQNAgent** <br />
Feel free to play with the nb_steps_warump, target_model_update, batch_size and gamma parameters. <br />
Hint:<br />
You can try *nb_steps_warmup*=1000, *target_model_update*=1000, *batch_size*=32 and *gamma*=0.99 as a first guess

In [10]:
dqn = DQNAgent(model=model, nb_actions=num_actions, memory=memory, 
               nb_steps_warmup=1000, target_model_update=1000, policy=policy,
               gamma=0.99, batch_size=32
              )

**TASK: Compile the model** <br />
Feel free to explore the effects of different optimizers and learning rates.
You can try Adam with a learning rate of 1e-3 as a first guess 

In [11]:
dqn.compile(Adam(learning_rate=1e-3), metrics=["mae"])

**TASK: Fit the model** <br />
150,000 steps should be a very good starting point

In [12]:
dqn.fit(env, nb_steps=150000, visualize=False, verbose=2)

Training for 150000 steps ...


C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\site-packages\keras\engine\training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


    500/150000: episode: 1, duration: 1.548s, episode steps: 500, steps per second: 323, episode reward: -500.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.024 [0.000, 2.000],  loss: --, mae: --, mean_q: --, mean_eps: --
   1000/150000: episode: 2, duration: 0.774s, episode steps: 500, steps per second: 646, episode reward: -500.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.002 [0.000, 2.000],  loss: --, mae: --, mean_q: --, mean_eps: --


C:\Users\VictorHernandez-Urbi\anaconda3\envs\env\lib\site-packages\keras\engine\training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


   1500/150000: episode: 3, duration: 6.676s, episode steps: 500, steps per second:  75, episode reward: -500.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.004 [0.000, 2.000],  loss: 0.007188, mae: 0.512267, mean_q: -0.701002, mean_eps: 0.992500
   2000/150000: episode: 4, duration: 5.994s, episode steps: 500, steps per second:  83, episode reward: -500.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.046 [0.000, 2.000],  loss: 0.000415, mae: 0.502049, mean_q: -0.718353, mean_eps: 0.989503
   2500/150000: episode: 5, duration: 5.957s, episode steps: 500, steps per second:  84, episode reward: -500.000, mean reward: -1.000 [-1.000, -1.000], mean action: 0.982 [0.000, 2.000],  loss: 0.008334, mae: 1.256795, mean_q: -1.825103, mean_eps: 0.986503
   3000/150000: episode: 6, duration: 6.069s, episode steps: 500, steps per second:  82, episode reward: -500.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.016 [0.000, 2.000],  loss: 0.000755, mae: 1.254379, mean_q

...

 135389/150000: episode: 643, duration: 1.855s, episode steps: 128, steps per second:  69, episode reward: -127.000, mean reward: -0.992 [-1.000,  0.000], mean action: 1.023 [0.000, 2.000],  loss: 0.624339, mae: 22.497711, mean_q: -32.645699, mean_eps: 0.188053
 135519/150000: episode: 644, duration: 1.928s, episode steps: 130, steps per second:  67, episode reward: -129.000, mean reward: -0.992 [-1.000,  0.000], mean action: 1.069 [0.000, 2.000],  loss: 0.577864, mae: 22.559071, mean_q: -32.709139, mean_eps: 0.187279
 135603/150000: episode: 645, duration: 1.278s, episode steps:  84, steps per second:  66, episode reward: -83.000, mean reward: -0.988 [-1.000,  0.000], mean action: 0.988 [0.000, 2.000],  loss: 0.628098, mae: 22.434583, mean_q: -32.504304, mean_eps: 0.186637
 135687/150000: episode: 646, duration: 1.253s, episode steps:  84, steps per second:  67, episode reward: -83.000, mean reward: -0.988 [-1.000,  0.000], mean action: 1.036 [0.000, 2.000],  loss: 0.669871, mae: 22.7

 139112/150000: episode: 675, duration: 1.646s, episode steps: 130, steps per second:  79, episode reward: -129.000, mean reward: -0.992 [-1.000,  0.000], mean action: 1.231 [0.000, 2.000],  loss: 0.734915, mae: 22.168101, mean_q: -32.093348, mean_eps: 0.165721
 139208/150000: episode: 676, duration: 1.192s, episode steps:  96, steps per second:  81, episode reward: -95.000, mean reward: -0.990 [-1.000,  0.000], mean action: 1.021 [0.000, 2.000],  loss: 0.668885, mae: 22.243820, mean_q: -32.293613, mean_eps: 0.165043
 139347/150000: episode: 677, duration: 1.773s, episode steps: 139, steps per second:  78, episode reward: -138.000, mean reward: -0.993 [-1.000,  0.000], mean action: 1.165 [0.000, 2.000],  loss: 0.665309, mae: 22.408902, mean_q: -32.471414, mean_eps: 0.164338
 139520/150000: episode: 678, duration: 2.274s, episode steps: 173, steps per second:  76, episode reward: -172.000, mean reward: -0.994 [-1.000,  0.000], mean action: 1.156 [0.000, 2.000],  loss: 0.643340, mae: 22.

 142390/150000: episode: 707, duration: 1.452s, episode steps: 118, steps per second:  81, episode reward: -117.000, mean reward: -0.992 [-1.000,  0.000], mean action: 0.780 [0.000, 2.000],  loss: 0.736946, mae: 22.330822, mean_q: -32.335937, mean_eps: 0.146017
 142489/150000: episode: 708, duration: 1.268s, episode steps:  99, steps per second:  78, episode reward: -98.000, mean reward: -0.990 [-1.000,  0.000], mean action: 0.909 [0.000, 2.000],  loss: 0.651389, mae: 22.475500, mean_q: -32.568982, mean_eps: 0.145366
 142573/150000: episode: 709, duration: 1.042s, episode steps:  84, steps per second:  81, episode reward: -83.000, mean reward: -0.988 [-1.000,  0.000], mean action: 0.940 [0.000, 2.000],  loss: 0.722776, mae: 22.071967, mean_q: -31.973662, mean_eps: 0.144817
 142673/150000: episode: 710, duration: 1.248s, episode steps: 100, steps per second:  80, episode reward: -99.000, mean reward: -0.990 [-1.000,  0.000], mean action: 0.740 [0.000, 2.000],  loss: 0.636538, mae: 22.22

 145742/150000: episode: 739, duration: 1.840s, episode steps: 122, steps per second:  66, episode reward: -121.000, mean reward: -0.992 [-1.000,  0.000], mean action: 0.590 [0.000, 2.000],  loss: 0.629177, mae: 22.734914, mean_q: -32.894172, mean_eps: 0.125917
 145832/150000: episode: 740, duration: 1.044s, episode steps:  90, steps per second:  86, episode reward: -89.000, mean reward: -0.989 [-1.000,  0.000], mean action: 0.956 [0.000, 2.000],  loss: 0.610255, mae: 22.711421, mean_q: -32.903394, mean_eps: 0.125281
 145927/150000: episode: 741, duration: 1.123s, episode steps:  95, steps per second:  85, episode reward: -94.000, mean reward: -0.989 [-1.000,  0.000], mean action: 0.758 [0.000, 2.000],  loss: 0.590614, mae: 22.528282, mean_q: -32.642552, mean_eps: 0.124726
 146062/150000: episode: 742, duration: 1.648s, episode steps: 135, steps per second:  82, episode reward: -134.000, mean reward: -0.993 [-1.000,  0.000], mean action: 0.615 [0.000, 2.000],  loss: 0.734329, mae: 22.3

 148725/150000: episode: 771, duration: 0.955s, episode steps:  86, steps per second:  90, episode reward: -85.000, mean reward: -0.988 [-1.000,  0.000], mean action: 1.198 [0.000, 2.000],  loss: 0.683938, mae: 22.256199, mean_q: -32.188518, mean_eps: 0.107911
 148810/150000: episode: 772, duration: 0.947s, episode steps:  85, steps per second:  90, episode reward: -84.000, mean reward: -0.988 [-1.000,  0.000], mean action: 1.082 [0.000, 2.000],  loss: 0.612877, mae: 22.238542, mean_q: -32.176607, mean_eps: 0.107398
 148901/150000: episode: 773, duration: 1.041s, episode steps:  91, steps per second:  87, episode reward: -90.000, mean reward: -0.989 [-1.000,  0.000], mean action: 1.176 [0.000, 2.000],  loss: 0.619375, mae: 22.170071, mean_q: -32.067989, mean_eps: 0.106870
 148969/150000: episode: 774, duration: 0.787s, episode steps:  68, steps per second:  86, episode reward: -67.000, mean reward: -0.985 [-1.000,  0.000], mean action: 1.103 [0.000, 2.000],  loss: 0.719771, mae: 22.269

**TASK: Evaluate the model**

In [14]:
dqn.test(env, nb_episodes=5, visualize=True)
env.close()

Testing for 5 episodes ...
Episode 1: reward: -500.000, steps: 500
Episode 2: reward: -114.000, steps: 115
Episode 3: reward: -500.000, steps: 500
Episode 4: reward: -500.000, steps: 500
Episode 5: reward: -500.000, steps: 500
